### Common Setup

In [43]:
import requests
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)

from matplotlib import pyplot as plt
%pylab inline

import base64
import json
import time
import urllib


hostname='bjosm02.dli.com'
dlpd_port='55557'
dlpd_rest_base_url = 'http://{}:{}'.format(hostname, dlpd_port) # http or https
print('DLPD_REST_BASE_URL_1: {}'.format(dlpd_rest_base_url))

commonHeaders={'accept': 'application/json'}
req = requests.Session()

Populating the interactive namespace from numpy and matplotlib
DLPD_REST_BASE_URL_1: http://bjosm02.dli.com:55557


### Health Check

Check if there is any existing hpo algorithms and also verify the platform health

Rest API: **GET platform/rest/deeplearning/v1/hypersearch/algorithm**
- Description: Get all the hpo algorithm that the login user can access.
- OUTPUT: A list of hpo algorithms and each one with the same format which can be found in the api doc.

In [47]:
getAllAlogUrl = '{}/platform/rest/deeplearning/v1/hypersearch/algorithm'.format(dlpd_rest_base_url)
r = req.get(getAllAlogUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
if not r.ok:
    print('check hpo algorithm failed: code=%s, %s'%(r.status_code, r.text))
else:
    if len(r.json()) == 0:
        print('There is no hpo algorithm been created')
    for item in r.json():
        print('Hpo alogrithm: %s, Type: %s'%(item['name'], item['type']))

Hpo alogrithm: Random, Type: BUILD_IN
Hpo alogrithm: TPE, Type: BUILD_IN
Hpo alogrithm: Bayesian, Type: BUILD_IN
Hpo alogrithm: Hyperband, Type: BUILD_IN
Hpo alogrithm: ExperimentGridSearch, Type: BUILD_IN
Hpo alogrithm: test-random-remote, Type: USER_PLUGIN
Hpo alogrithm: test-random-info, Type: USER_PLUGIN


### Install a HPO user plugin algorithm

There are 2 methods are supported for the plugin algorithm installation:
- Local install: Use this way if the plugin algorithm scripts are allocated in the dlpd server, or in a shared folder that dlpd service can access.
- Upload install: Use this way if you want to upload the plugin algorithm scrips from a client host.

#### Local install plugin algorithm

REST API: POST **/platform/rest/deeplearning/v1/hypersearch/algorithm/install**

Description: Install a new hpo plugin algorithm.  
Content-type: Multi-Form

Multi-Form Data:  
- form-filed: {‘data’: ‘String json format of input parameters to install a hpo plugin algorithm’}


##### Construct the POST request body

In [48]:
create_algorithm_name = 'test-random-debug'
data = {
    # required, string, name/id for the plugin algorithm, should be unique.,
    "name": create_algorithm_name,
    
    # required for local install, string, the path for plugin algorithm scripts on server.
    "path": "/opt/zhuangxy/hporfe/debug_work_dir/random_plugin_example/", 
    
    # optional, string, the CONDA_HOME to run the algorithm scripts.
    # if not specified, it will use the DLI_CONDA_HOME.
    "condaHome": "/opt/anaconda3", 
    
    # optional, string, the conda environment to run the algorithm scripts.
    # if not specified, it will use the DLI default conda environment (dlipy3).
    "condaEnv": "hpo", 
    
    # optional, boolean, whether to deploy algorithm execution remotely, the default value is false.
    "remoteExec": False, 
    
    # optional, string, the log level of the plugin algorithm, the default value is INFO.
    "logLevel": "DEBUG"
    }
mydata={'data':json.dumps(data)}

##### Submit the installtion request

In [49]:
installPluginAlgoURL = '{}/platform/rest/deeplearning/v1/hypersearch/algorithm/install'.format(dlpd_rest_base_url)
install = req.post(installPluginAlgoURL, 
                   headers=commonHeaders, 
                   data=mydata, 
                   files=mydata, 
                   verify=False, 
                   auth=('Admin', 'Admin'))
if not install.ok:
    print('install plugin algorithm failed: code=%s, %s' % (install.status_code, install.text))
else:
    # query the new created plugin algorithm
    getAlogUrl = '{}/platform/rest/deeplearning/v1/hypersearch/algorithm/{}'.format(
        dlpd_rest_base_url, create_algorithm_name)
    algoDesc = req.get(getAlogUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
    print('install plugin algorithm success: \n%s' % json.dumps(algoDesc.json(), indent=4))


install plugin algorithm success: 
{
    "name": "test-random-debug",
    "path": "/opt/zhuangxy/hporfe/share_fs/tools/tune/plugins/test-random-debug",
    "condaHome": "/opt/anaconda3",
    "condaEnv": "hpo",
    "creator": "Admin",
    "createtime": "2020-01-09 17:31:32",
    "type": "USER_PLUGIN",
    "remoteExec": false,
    "logLevel": "DEBUG"
}


#### Upload install plugin algorithm

REST API: POST **/platform/rest/deeplearning/v1/hypersearch/algorithm/install**

Description: Install a new hpo plugin algorithm.  
Content-type: Multi-Form

Multi-Form Data:
- files: plugin algorithm scripts tar package, ending with .tar
- form-filed: {'data': 'String json format of input parameters to install a hpo plugin algorithm'}

##### Prepare the upload tar file

In [112]:
import tarfile
import tempfile
import os
def make_tarfile(output_filename, source_dir):
    print('source dir: ' + source_dir)
    with tarfile.open(output_filename, "w:gz") as tar:
        for root, _, files in os.walk(source_dir):  
            for file in files:  
                fullpath=os.path.join(root, file) 
                print('tar added: ' + fullpath)
                tar.add(fullpath, arcname=file)
        
MODEL_DIR_SUFFIX = ".tar"
tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)
make_tarfile(tempFile, '/opt/zhuangxy/hporfe/debug_work_dir/random_plugin_example/')
print('plugin temp tar file: {}'.format(tempFile))
files = {'file': open(tempFile, 'rb')}

source dir: /opt/zhuangxy/hporfe/debug_work_dir/random_plugin_example/
tar added: /opt/zhuangxy/hporfe/debug_work_dir/random_plugin_example/optimizer.py
plugin temp tar file: /tmp/tmp0j5_leuf.tar


##### Construct the POST request body

In [113]:
create_algorithm_name = 'test-random-uploadc'
data = {
    # required, string, name/id for the plugin algorithm, should be unique.,
    "name": create_algorithm_name,
    
    # optional, string, the CONDA_HOME to run the algorithm scripts.
    # if not specified, it will use the DLI_CONDA_HOME.
    "condaHome": "/opt/anaconda3", 
    
    # optional, string, the conda environment to run the algorithm scripts.
    # if not specified, it will use the DLI default conda environment (dlipy3).
    "condaEnv": "hpo", 
    
    # optional, boolean, whether to deploy algorithm execution remotely, the default value is false.
    "remoteExec": False, 
    
    # optional, string, the log level of the plugin algorithm, the default value is INFO.
    "logLevel": "DEBUG"
    }
mydata={'data':json.dumps(data)}

##### Submit the installtion request

In [114]:
installPluginAlgoURL = '{}/platform/rest/deeplearning/v1/hypersearch/algorithm/install'.format(dlpd_rest_base_url)
install = req.post(installPluginAlgoURL, 
                   headers=commonHeaders, 
                   data=mydata, 
                   files=files, 
                   verify=False, 
                   auth=('Admin', 'Admin'))
if not install.ok:
    print('install plugin algorithm failed: code=%s, %s' % (install.status_code, install.text))
else:
    # query the new created plugin algorithm
    getAlogUrl = '{}/platform/rest/deeplearning/v1/hypersearch/algorithm/{}'.format(
        dlpd_rest_base_url, create_algorithm_name)
    algoDesc = req.get(getAlogUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
    print('install plugin algorithm success: \n%s' % json.dumps(algoDesc.json(), indent=4))
    os.remove(tempFile)


install plugin algorithm success: 
{
    "name": "test-random-uploadc",
    "path": "/opt/zhuangxy/hporfe/share_fs/tools/tune/plugins/test-random-uploadc",
    "condaHome": "/opt/anaconda3",
    "condaEnv": "hpo",
    "creator": "Admin",
    "createtime": "2020-01-08 17:11:29",
    "type": "USER_PLUGIN",
    "remoteExec": false,
    "logLevel": "DEBUG"
}


### Delete a HPO user plugin algorithm

REST API: **DELETE /platform/rest/deeplearning/v1/hypersearch**

Description: delete a hpo plugin algorithm by algorithm name.  


In [45]:
delete_algorithm_name = 'test-random-debug'
deleteAlogUrl = '{}/platform/rest/deeplearning/v1/hypersearch/algorithm/{}'.format(
        dlpd_rest_base_url, delete_algorithm_name)
delete = req.delete(deleteAlogUrl, 
                   headers=commonHeaders, 
                   verify=False, 
                   auth=('Admin', 'Admin'))
if not delete.ok:
    print('delete plugin algorithm failed: code=%s, %s' % (delete.status_code, delete.text))
else:
    print('delete plugin algorithm {} success'.format(delete_algorithm_name))

delete plugin algorithm test-random-debug success


### Query a HPO user plugin algorithm

#### Query a HPO user plugin algorithm by name

Rest API: **GET platform/rest/deeplearning/v1/hypersearch/algorithm/{algoName}**
- Description: Get a hpo algorithm by name.
- OUTPUT: A hpo algorithm with the same format which can be found in the api doc.  

**Full Response of a hpo algorithm description:**
```
HpoAlgorithmDesc {
    name (string): 'The hpo algorithm name.' ,
    path (string, optional): 'The hpo algorithm installation path. (only for plugin algorithm).' ,
    condaHome (string, optional): 'The CONDA_HOME used to run hpo algorithm (only for plugin algorithm).' ,
    condaEnv (string, optional): 'The conda environment used to run hpo algorithm (only for plugin algorithm).' ,
    createtime (string, optional): 'The creation time of the hpo algorithm (only for plugin algorithm).' ,
    type (string, optional): 'The type of the hpo algorithm.' ,
    remoteExec (boolean, optional): 'The plugin algorithm execution mode is remoted or not (only for plugin algorithm).' ,
    logLevel (string, optional): 'The log level for the plugin algorithm (only for plugin algorithm)'.
}
```

In [131]:
query_algorithm_name = 'test-random-remote'
getAlogUrl = '{}/platform/rest/deeplearning/v1/hypersearch/algorithm/{}'.format(
        dlpd_rest_base_url, query_algorithm_name)
query = req.get(getAlogUrl, 
                   headers=commonHeaders, 
                   verify=False, 
                   auth=('Admin', 'Admin'))
if not query.ok:
    print('query plugin algorithm failed: code=%s, %s' % (query.status_code, query.text))
else:
    if not query.json():
        print('plugin algorithm {} does not exist'.format(query_algorithm_name))
    else:
        print('query plugin algorithm success: {}'.format(json.dumps(query.json(), indent=4)))

query plugin algorithm success: {
    "name": "test-random-remote",
    "path": "/opt/zhuangxy/hporfe/share_fs/tools/tune/plugins/test-random-remote",
    "condaHome": "/opt/anaconda3",
    "condaEnv": "hpo",
    "creator": "Admin",
    "createtime": "2019-12-27 15:47:20",
    "type": "USER_PLUGIN",
    "remoteExec": true
}


#### Query a HPO user plugin algorithm by type

Rest API: **GET platform/rest/deeplearning/v1/hypersearch/algorithm?type={algo_type}**
- Description: Get hpo algorithms by type. If the type is not specified, all hpo algorithms will be returned.
- OUTPUT: A list hpo algorithms and each one with the same format which can be found in the api doc.  

In [135]:
algo_type = 'BUILD_IN'
# algo_type = 'USER_PLUGIN'
getAlogUrl = '{}/platform/rest/deeplearning/v1/hypersearch/algorithm?type={}'.format(
        dlpd_rest_base_url, algo_type)
query = req.get(getAlogUrl, 
                   headers=commonHeaders, 
                   verify=False, 
                   auth=('Admin', 'Admin'))
if not query.ok:
    print('query plugin algorithm failed: code=%s, %s' % (query.status_code, query.text))
else:
    if not query.json():
        print('plugin algorithm {} does not exist'.format(query_algorithm_name))
    else:
        print('query plugin algorithm success: {}'.format(json.dumps(query.json(), indent=4)))

query plugin algorithm success: [
    {
        "name": "Random",
        "type": "BUILD_IN",
        "remoteExec": false
    },
    {
        "name": "TPE",
        "type": "BUILD_IN",
        "remoteExec": false
    },
    {
        "name": "Bayesian",
        "type": "BUILD_IN",
        "remoteExec": false
    },
    {
        "name": "Hyperband",
        "type": "BUILD_IN",
        "remoteExec": false
    },
    {
        "name": "ExperimentGridSearch",
        "type": "BUILD_IN",
        "remoteExec": false
    }
]


### Launch a HPO task

#### Model file update to Run HPO

Model changes required from 2 perspective:
- Inject hyper-parameters for the sub-training during search
- Retrieve sub-training result metric

##### Model update part 1 - Inject hyper-parameters

The hyper-parameters will be supplied in a file called **config.json** with JSON format,located in the current working directory and can be read direcly as the following example snippet.

<pre>
hyper_params = json.loads(open("<b>config.json</b>").read())
learning_rate = float(hyper_params.get("<b>learning_rate</b>", "0.01"))
</pre>

After this, you can use these hyper-parameters during the model trainings. The **hyper-parameter name** and **value** type is defined through the search space part in body of REST call when launching a new hpo task.

##### Model update part 2 - Retrieve sub-training result metric

At the end of your training run, your code will need to create a file called **val_dict_list.json** with test metrics generated during training. These metrics will be used by the search algorithm to propose new sets of hyper-parameters. Please note that **val_dict_list.json** should be created under the result directory which can be retrieved through the environment variable **RESULT_DIR**.

<pre>
with open('{}/val_dict_list.json'.format(os.environ['<b>RESULT_DIR</b>']), 'w') as f:
    json.dump(test_metrics, f)
</pre>

The content of **val_dict_list.json** will be some thing as below, **step** is some thing optional meaning the training iteration or epochs, one of **loss** and **accuracy** can be the name of target metric to optimize, at least one metric need to be included here. The specific name of metric used to optimize (minimize or maximize) is defined in the body of REST call when launching a new hpo task. 

```
[
{‘step’: 1, ‘loss’:0.2487, ‘accuracy’: 0.4523},
{‘step’: 2, ‘loss’:0.1487, ‘accuracy’: 0.5523},
{‘step’: 3, ‘loss’:0.1087, ‘accuracy’: 0.6523},
…
]
```

#### Launch HPO task

REST API: **POST /platform/rest/deeplearning/v1/hypersearch**
- Description: Start a new HPO task
- Content-type: Multi-Form
- Multi-Form Data:
  - files: Model files tar package, ending with `.modelDir.tar`
  - form-filed: {‘data’: ‘String format of input parameters to start hpo task, let’s call it as **hpo_input** and show its specification later’}


##### Package model files for training

Package the updated model files into a tar file ending with `.modelDir.tar`

In [50]:
import tarfile
import tempfile
import os
def make_tarfile(output_filename, source_dir):
    with tarfile.open(output_filename, "w:gz") as tar:
        tar.add(source_dir, arcname=os.path.basename(source_dir))
MODEL_DIR_SUFFIX = ".modelDir.tar"

tempFile = tempfile.mktemp(MODEL_DIR_SUFFIX)
make_tarfile(tempFile, '/opt/zhuangxy/hporfe/tf-model')
files = {'file': open(tempFile, 'rb')}
print("tar model files: " + tempFile)

tar model files: /tmp/tmph1fcrdqy.modelDir.tar


##### Construct POST request data

**hpo_input** will be a Python dict or json format as below, convert to string when calling REST.

In [51]:
data =  {
        'modelSpec': # Define the model training related parameters
        {
            # Spark instance group which will be used to run the HPO sub-trainings. The Spark instance group selected
            # here should match the sub-training args, for example, if the sub-training args try to run a EDT job,
            # then we should put a Spark instance group with capability to run EDT job here.
            'sigName': 'sig233',
            
            # These are the arguments we'll pass to the execution engine; they follow the same conventions
            # of the dlicmd.py command line launcher
            #
            # See:
            #   https://www.ibm.com/support/knowledgecenter/en/SSFHA8_1.2.1/cm/dlicmd.html
            # In this example, args after --model-dir are all the required parameter for the original model itself.
            'args': '--exec-start tensorflow --python-version 2.7 --cs-datastore-meta type=fs \
                     --gpuPerWorker 1 --model-main convolutional_network.py --model-dir tf-model\
                     --trainImagesFile /opt/zhuangxy/hporfe/tf-mnist/train-images-idx3-ubyte.gz \
                     --trainLabelsFile /opt/zhuangxy/hporfe/tf-mnist/train-labels-idx1-ubyte.gz \
                     --testImagesFile /opt/zhuangxy/hporfe/tf-mnist/t10k-images-idx3-ubyte.gz \
                     --testLabelsFile /opt/zhuangxy/hporfe/tf-mnist/t10k-labels-idx1-ubyte.gz'
        },
    
        'algoDef': # Define the parameters for search algorithms
        {
            # Name of the plugin algorithm
            'algorithm': 'test-random-debug', 
            # Max running time of the hpo task in minutes, -1 means unlimited
            'maxRunTime': 60,  
            # Max number of training job to submitted for hpo task, -1 means unlimited’,
            'maxJobNum': 3,            
            # Max number of training job to run in parallel, default 1. It depends on both the
            # avaiable resource and if the search algorithm support to run in parallel, current only Random
            # fully supports to run in parallel, Hyperband and Tpe supports to to in parellel in some phase,
            # Bayesian runs in sequence now.
            'maxParalleJobNum': 1, 
            # Name of the target metric that we are trying to optimize when searching hyper-parameters.
            # It is the same metric name that the model update part 2 trying to dump.
            'objectiveMetric' : 'loss',
            # Strategy as how to optimize the hyper-parameters, minimize means to find better hyper-parameters to
            # make the above objectiveMetric as small as possible, maximize means the opposite.
            'objective' : 'minimize',
            #Additional parameters for the specified search algorithm and hyper-band get following too.
            'algoParams' : 
                [
                    {
                        # parameters passed to the plugin algorithm
                        'name':'random_seed', 
                        'value': '2'
                    }
                ]
        },
    
        # Define the hyper-paremeters to search and the corresponding search space.
        'hyperParams':
        [
             {
                 # Hyperparameter name, which will be the hyper-parameter key in config.json
                 'name': 'learning_rate',
                 # One of Range, Discrete
                 'type': 'Range',
                 # one of int, double, str
                 'dataType': 'DOUBLE',
                 # lower bound and upper bound when type=range and dataType=double
                 'minDbVal': 0.001,
                 'maxDbVal': 0.1,
                 # lower bound and upper bound when type=range and dataType=int
                 'minIntVal': 0,
                 'maxIntVal': 0,
                 # Discrete value list when type=discrete
                 'discreteDbVal': [],
                 'discreteIntVal': [],
                 'discreateStrVal': [],
                 #step size to split the Range space. ONLY valid when type is Range
                 'step': '0.002',
             }
         ]
    }
mydata={'data':json.dumps(data)}

##### Submit the Post request

Submit hpo task through the Post call and a hpo name/id as string format will get back.

In [52]:
startTuneUrl='{}/platform/rest/deeplearning/v1/hypersearch'.format(dlpd_rest_base_url)
create = req.post(startTuneUrl, headers=commonHeaders, data=mydata, files=files, verify=False, auth=('Admin', 'Admin'))
if not create.ok:
   print('submit tune job failed: code=%s, %s'%(create.status_code, create.content))
else:
   print('submit tune job succeed with hponame: %s'%create.json())

submit tune job succeed with hponame: Admin-hpo-1132672298136969


### Check HPO task status

REST API: **GET /platform/rest/deeplearning/v1/hypersearch/{hponame}**
- Description: Retrieve the hpo task details with the specified hpo task name/id in URL.
- OUTPUT: A particular hpo task with details of the specified hponame

In [53]:
import time
#Here we just use the hpo name returned of the previous POST call
hpoName = create.json()

getHpoUrl = '{}/platform/rest/deeplearning/v1/hypersearch/{}'.format(dlpd_rest_base_url, hpoName)
res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
if not res.ok:
    print('get hpo task failed: code=%s, %s'%(res.status_code, res.content))
else:
    json_out=res.json()
    
    while json_out['state'] in ['SUBMITTED','RUNNING']:
        print('Hpo task %s state %s progress %s'%(hpoName, json_out['state'], json_out['progress']))
        res = req.get(getHpoUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
        json_out=res.json()
        time.sleep(20)
    
    print('Hpo task %s completes with state %s'%(hpoName, json_out['state']))
    #print('Best:%s'%json.dumps(json_out.get('best'), sort_keys=True, indent=4))
    print(json.dumps(res.json(), sort_keys=True, indent=4))

Hpo task Admin-hpo-1132672298136969 state RUNNING progress 0/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 0/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 0/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 0/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 0/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 1/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 1/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 1/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 1/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 2/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 2/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 2/3
Hpo task Admin-hpo-1132672298136969 state RUNNING progress 2/3
Hpo task Admin-hpo-1132672298136969 completes with state FINISHED
{
    "best": {
        "appId": "Admin-1132760534098217-828876449",
        "driverId": "driver-20200109173332-004

**Full Response of a hpo task status.**

```
{
"hpoName": "string, name/id of the hpo task",
"state": "string, hpo task state, SUBMITTED, RUNNING, FAILED, FINISHED, STOPPED",
"running": "int, number of training that is under-going right now",
"complete": "int, number of training that has completes, including both succeeded and failed trainings",
"failed": "int, number of training that failed",
"progress": "string, progress of a percentage value",
"createtime": "string, time this task was created",
"creator": "string, user name who created this hpo task",
"duration": "string, how long the task been run with format hh-mm-ss",
"experiments": [
                {
                "id": "int, counter id of the experiment training, start from 0",
                "metricVal": "double, best metric value of the experiment training",
                "state": "string, state of the BYOF training task"
                "appId": "string, BYOF training task id",
                "driverId": "string, for internal usage, the real job id for training, currently it is the spark driver id",
                "hyperParams": [
                                {
                                "name": "string, name of the hyperparameter",
                                "dataType": "string, data type of the hyperparameter",
                                "fixedVal": "string, The hyperparamter value been used in this experiment training with the same datatype as input dataType"
                                }
                                ],
                }
                ],
"best": {"one of the experiments with the best metric value, smallest or biggest one based on the original objective minimize or maximize"}
}

```


### Manage HPO tasks

#### Check HPO tasks

Same REST api as the early health check one.

In [54]:
getTuneStatusUrl = '{}/platform/rest/deeplearning/v1/hypersearch'.format(dlpd_rest_base_url)
res = req.get(getTuneStatusUrl, headers=commonHeaders, verify=False, auth=('Admin', 'Admin'))
if not res.ok:
    print('check tune job status failed: code=%s, %s'%(res.status_code, res.content))
else:
    #print(json.dumps(r.json(), sort_keys=True, indent=4))
    if len(res.json()) == 0:
        print('There is no hpo task been created')
    for item in res.json():
        #print(item['hpoName'])
        print('Hpo task: %s, State: %s'%(item['hpoName'], item['state']))
        #print('Hpo tasks detail:%s'%json.dumps(item, sort_keys=True, indent=4))

Hpo task: Admin-hpo-1132672298136969, State: FINISHED


#### Stop Hpo task

REST API: **PUT /platform/rest/deeplearning/v1/hypersearch/{hponame}**
- Description: Stop the specified hpo job with name/di {hponame}
- OUTPUT: success http return code

Since the hpo job has already finished, so nothing to stop here.

In [55]:
stopHpoUrl = '{}/platform/rest/deeplearning/v1/hypersearch/{}'.format(dlpd_rest_base_url, hpoName)
r=req.put(stopHpoUrl,headers=commonHeaders, verify=False, auth=('Admin','Admin'))
if not r.ok:
    print('stop hpo task failed: code=%s, %s'%(r.status_code, r.content))
else:
    print('stop hpo task succeeds')

stop hpo task failed: code=400, b'Error 400: Model tuning Admin-hpo-1132672298136969 is already stopped.\n'


#### Delete Hpo tasks

REST API: **DELETE /platform/rest/deeplearning/v1/hypersearch/{hponame}**
- Description: Delete the specified hpo task with name/id {hponame}
- OUTPUT: success http return code

REST API: **DELETE /platform/rest/deeplearning/v1/hypersearch**
- Description: Delete all the hpo task that the login user can access. But please be aware, the BYOF training been created by this hpo task will not be deleted.
- OUTPUT: success http return code

In [40]:
deleteHpoUrl = '{}/platform/rest/deeplearning/v1/hypersearch'.format(dlpd_rest_base_url)
r=req.delete(deleteHpoUrl,headers=commonHeaders, verify=False, auth=('Admin','Admin'))
if not r.ok:
    print('delete hpo task failed: code=%s, %s'%(r.status_code, r.content))
else:
    print('delete hpo task succeeds')

delete hpo task succeeds
